# Whiskey Data

This data set contains data on a small number of whiskies

In [2]:
import pandas as pd
from numpy import log, abs, sign, sqrt
import brunel

data = pd.read_csv("data/whiskey.csv")

print('Data on whiskies:', ', '.join(data.columns))

ImportError: No module named 'brunel'

## Summaries
Shown below are the following charts:

 * A treemap display for each whiskey, broken down by _country_ and _category_. The cells are colored by the rating, with lower-rated whiskies in blue, and higher-rated in reds. Missing data for ratings show as black.
 * A filtered chart allowing you to select whiskeys based on price and category
 * A line chart showing the relationship between _age_ and _rating_. A simple treemap of categories is linked to this chart
 * A bubble chart of countries linked to a heatmap of alcohol level (_ABV_) by _rating_

In [1]:
%brunel x(country, category) color(rating) treemap label(name:3) tooltip(#all) style('.label {font-size:7pt}') legends(none):: width=900, height=600

ERROR: Line magic function `%brunel` not found.


In [3]:
%brunel bubble color(rating:red) sort(rating) size(abv) label(name:6) tooltip(#all) filter(price, category) :: height=500

<!--
 ~ Copyright (c) 2015 IBM Corporation and others.
 ~
 ~ Licensed under the Apache License, Version 2.0 (the "License");
 ~ You may not use this file except in compliance with the License.
 ~ You may obtain a copy of the License at
 ~
 ~ http://www.apache.org/licenses/LICENSE-2.0
 ~
 ~ Unless required by applicable law or agreed to in writing, software
 ~ distributed under the License is distributed on an "AS IS" BASIS,
 ~ WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 ~ See the License for the specific language governing permissions and
 ~ limitations under the License.
 ~
 -->

In [4]:
%%brunel 
line x(age) y(rating) mean(rating) label(country)  using(interpolate) bin(age:8) color(#selection) legends(none) 
| treemap x(category) interaction(select) size(#count) color(#selection) legends(none) sort(#count:ascending) bin(category:9)
        tooltip(country) list(country) label(#count) style('.labels .label {font-size:14px}')
:: width=900

<!--
 ~ Copyright (c) 2015 IBM Corporation and others.
 ~
 ~ Licensed under the Apache License, Version 2.0 (the "License");
 ~ You may not use this file except in compliance with the License.
 ~ You may obtain a copy of the License at
 ~
 ~ http://www.apache.org/licenses/LICENSE-2.0
 ~
 ~ Unless required by applicable law or agreed to in writing, software
 ~ distributed under the License is distributed on an "AS IS" BASIS,
 ~ WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 ~ See the License for the specific language governing permissions and
 ~ limitations under the License.
 ~
 -->

<IPython.core.display.Javascript object>

In [5]:
%%brunel 
bubble label(country:3) bin(country) size(#count) color(#selection) sort(#count) interaction(select) tooltip(name) list(name) legends(none)
| x(abv) y(rating) color(#count:blue) legends(none) bin(abv:8) bin(rating:5) style('symbol:rect; stroke:none; size:100%')  at(0,10,70,100)
        interaction(select) label(#selection) list(#selection)  at(60,15,100,100) tooltip(rating, abv,#count) legends(none) 
|  bar label(brand:70) list(brand) at(0,0, 100, 10) axes(none) color(#selection) legends(none) interaction(filter)
:: width=900, height=600

<!--
 ~ Copyright (c) 2015 IBM Corporation and others.
 ~
 ~ Licensed under the Apache License, Version 2.0 (the "License");
 ~ You may not use this file except in compliance with the License.
 ~ You may obtain a copy of the License at
 ~
 ~ http://www.apache.org/licenses/LICENSE-2.0
 ~
 ~ Unless required by applicable law or agreed to in writing, software
 ~ distributed under the License is distributed on an "AS IS" BASIS,
 ~ WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 ~ See the License for the specific language governing permissions and
 ~ limitations under the License.
 ~
 -->

<IPython.core.display.Javascript object>

## Some Analysis
Here we use the sci-kit decision tree regression tool to predict the price of a whiskey given its age, rating and ABV value.
We transform the output for plotting purposes, but note that the tooltips give the original data

In [4]:
from sklearn import tree
D = data[['Name', 'ABV', 'Age', 'Rating', 'Price']].dropna()
X = D[ ['ABV', 'Age', 'Rating'] ]
y = D['Price']
clf = tree.DecisionTreeRegressor(min_samples_leaf=4)
clf.fit(X, y)
D['Predicted'] = clf.predict(X)
f = D['Predicted'] - D['Price']
D['Diff'] = sqrt(abs(f)) * sign(f)
D['LPrice'] = log(y)
%brunel y(diff) x(LPrice) tooltip(name, price, predicted, rating) color(rating)  :: width=700

<!--
 ~ Copyright (c) 2015 IBM Corporation and others.
 ~
 ~ Licensed under the Apache License, Version 2.0 (the "License");
 ~ You may not use this file except in compliance with the License.
 ~ You may obtain a copy of the License at
 ~
 ~ http://www.apache.org/licenses/LICENSE-2.0
 ~
 ~ Unless required by applicable law or agreed to in writing, software
 ~ distributed under the License is distributed on an "AS IS" BASIS,
 ~ WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 ~ See the License for the specific language governing permissions and
 ~ limitations under the License.
 ~
 -->

<IPython.core.display.Javascript object>

# Simple Linked Charts 

In [10]:
%%brunel bar x(country) y(#count)
    | bar color(category) y(#count) polar stack label(category) legends(none)
    :: width=900, height=300

<!--
 ~ Copyright (c) 2015 IBM Corporation and others.
 ~
 ~ Licensed under the Apache License, Version 2.0 (the "License");
 ~ You may not use this file except in compliance with the License.
 ~ You may obtain a copy of the License at
 ~
 ~ http://www.apache.org/licenses/LICENSE-2.0
 ~
 ~ Unless required by applicable law or agreed to in writing, software
 ~ distributed under the License is distributed on an "AS IS" BASIS,
 ~ WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 ~ See the License for the specific language governing permissions and
 ~ limitations under the License.
 ~
 -->

<IPython.core.display.Javascript object>